In [2]:
from ortools.sat.python import cp_model

import csv

import pandas as pd



# Declaring Parameters of model

NumStudents = 500

NumCourses = 20

NumTimeSlots = 4

MaxNumberDays = 60

RoomCapacity = 50

NumRooms = 6

# Dictionary (student, course). 1 if student takes the course, 0 otherwise

StudentCourse = {}



# Opening and reading csv file

with open('Course_Student_Association_Matrix.csv') as csvfile:

    readCSV = csv.reader(csvfile, delimiter=',')

    course = 0

    for row in readCSV:

        student = 0

        for cell in row:

        	if(cell != ''):

	        	StudentCourse[(course,student)] = int(cell)

	        	student = student + 1

        course = course + 1



# Defining Models

model = cp_model.CpModel()



# Binary Variable. 1 if exams from courses c1 and c2 are executed in same Day. 0 otherwise

SameDay = {}

for c1 in range(NumCourses):

	for c2 in range(NumCourses):

		if(c1 != c2):

			SameDay[(c1,c2)] = model.NewBoolVar('SameDay_' + str(c1) + str(c2))



# Binary Variable. 1 if exams from courses c1 and c2 are executed in same Time Slot. 0 otherwise

SameTimeSlot = {}

for c1 in range(NumCourses):

	for c2 in range(NumCourses):

		if(c1 != c2):

			SameTimeSlot[(c1,c2)] = model.NewBoolVar('SameTimeSlot_' + str(c1) + str(c2))



# Integer Variable: day which exam of course is executed

Day = [model.NewIntVar(0, MaxNumberDays - 1, 'Day%i' % i) for i in range(NumCourses)]



# Integer Variable: time slot which exam of course is executed

TimeSlot = [model.NewIntVar(0, NumTimeSlots - 1, 'TimeSlot%i' % i) for i in range(NumCourses)]



# Integer Variable: room which exam of course is executed

Room = [model.NewIntVar(0, NumRooms - 1, 'Room%i' % i) for i in range(NumCourses)]



# Constraint: exams cannot share same room at same day and time slot

for c1 in range(NumCourses):

	for c2 in range(NumCourses):

		if(c1 != c2):

			model.Add(Day[c1] == Day[c2]).OnlyEnforceIf(SameDay[c1,c2])

			model.Add(Day[c1] != Day[c2]).OnlyEnforceIf(SameDay[c1,c2].Not())

			model.Add(TimeSlot[c1] == TimeSlot[c2]).OnlyEnforceIf(SameTimeSlot[c1,c2])

			model.Add(TimeSlot[c1] != TimeSlot[c2]).OnlyEnforceIf(SameTimeSlot[c1,c2].Not())

			model.Add(Room[c1] != Room[c2]).OnlyEnforceIf(SameTimeSlot[c1,c2] and SameDay[c1,c2])



# Constraint: student cannot do an exam at the same time and day

for c1 in range(NumCourses):

	for c2 in range(NumCourses):

		if(c1 != c2):

			for s in range(NumStudents):

				if(StudentCourse[c1,s] == 1 and StudentCourse[c2,s] == 1):

					model.Add(Day[c1] != Day[c2] or TimeSlot[c1] != TimeSlot[c2])



# Defining the list of courses taken by each student

CoursesPerStudent = []

for s in range(NumStudents):

	Courses = []

	for c in range(NumCourses):

		if(StudentCourse[c,s] == 1):

			Courses.append(c)

	CoursesPerStudent.append(Courses)



# Constraint: A student cannot do more than two exams per day

for s in CoursesPerStudent:

	if(len(s) > 2):

		model.Add(sum(SameDay[c1,c2] for c1 in s for c2 in s if c1 > c2) <= 2)



# Constraint: If student takes 2 exams in a day, they cannot be in sequence

for s in CoursesPerStudent:

	if(len(s) > 2):

		for c1 in range(NumCourses):

			for c2 in range(NumCourses):

				if(c1 > c2):

					model.Add(TimeSlot[c1] != TimeSlot[c2] + 1).OnlyEnforceIf(SameDay[c1,c2])



# Objective function: Maximize the number of courses taken in same day - same as minimizing the number of days used

model.Maximize(sum(SameDay[c1,c2] for c1 in range(NumCourses) for c2 in range(NumCourses) if c1 > c2))

solver = cp_model.CpSolver()

status = solver.Solve(model)

Days_Used = []

if status == cp_model.OPTIMAL:		

	print('Optimal Solution Found')



for c in range(NumCourses):

	if(solver.Value(Day[c]) not in Days_Used):

		Days_Used.append(solver.Value(Day[c]))



# Number of days used

print("Number of Days Used", len(Days_Used))



for c in range(NumCourses):

	print("course ", c, "room ", solver.Value(Room[c]), "day ", solver.Value(Day[c]), "time slot ", solver.Value(TimeSlot[c]))



# for s in range(NumStudents):

# 	courses_taken = []

# 	for c in range(NumCourses):

# 		if(StudentCourse[c,s] == 1):

# 			courses_taken.append((solver.Value(Day[c]),solver.Value(TimeSlot[c])))

# 	print(s, courses_taken)



# Printing solution into dataframe and csv

courses = []

for i in range(NumCourses):

	courses.append(i)



output_opt_df = pd.DataFrame(index=courses,

                                 columns=["Room", "Day", "Time Slot"])



for i in courses:

	output_opt_df.loc[i] = [solver.Value(Room[i]), solver.Value(Day[i]), solver.Value(TimeSlot[i])]



output_opt_df.to_csv("Optimization_Solution.csv")

Optimal Solution Found
Number of Days Used 13
course  0 room  0 day  5 time slot  3
course  1 room  2 day  5 time slot  1
course  2 room  2 day  9 time slot  3
course  3 room  4 day  0 time slot  3
course  4 room  3 day  1 time slot  0
course  5 room  5 day  2 time slot  2
course  6 room  5 day  7 time slot  2
course  7 room  5 day  4 time slot  3
course  8 room  5 day  10 time slot  0
course  9 room  5 day  12 time slot  0
course  10 room  4 day  7 time slot  2
course  11 room  5 day  6 time slot  1
course  12 room  3 day  8 time slot  0
course  13 room  0 day  3 time slot  0
course  14 room  3 day  4 time slot  0
course  15 room  4 day  2 time slot  0
course  16 room  1 day  2 time slot  0
course  17 room  0 day  8 time slot  3
course  18 room  4 day  11 time slot  0
course  19 room  5 day  1 time slot  3
